In [1]:
import pandas
import os

parameters

In [2]:
median_min, median_max = pandas.Timestamp('2010-01-01'), pandas.Timestamp('2019-01-01')

x, y = 197691, 564057
d = 500

Function:
* combine hand an diver

In [3]:
def h(uuid_hand, uuid_diver):
    if uuid_hand!='' and uuid_diver=='':
        return pylizard.get_timeseries(uuid_hand)
    elif uuid_diver!='' and uuid_hand=='':
        return pylizard.get_timeseries(uuid_diver)
    else:
        h = pandas.concat([pylizard.get_timeseries(uuid_hand), pylizard.get_timeseries(uuid_diver)], axis=1)
        h.columns = ['hand', 'diver']
        
        h['head'] = h['diver']
        h.loc[h.index<h['diver'].dropna().index.min(), 'head'] = h.loc[h.index<h['diver'].dropna().index.min(), 'hand']
        
        return h['head']

Collect data

In [4]:
import pylizard

df = pylizard.pnt2buis(x, y, d)
df = df.loc[(df['uuid_hand']!='')|(df['uuid_diver']!='')]
df.index.name='id'

In [5]:
head = {}
median = {}
tmin = {}
tmax = {}
for i, r in df.iterrows():
    headi = h(r['uuid_hand'], r['uuid_diver'])
    head[i] = headi.loc[abs(headi-headi.median())<7.5*headi.std()]
    
    median[i] = head[i].loc[(head[i].index>median_min)&(head[i].index<median_max)].median()
    tmin[i] = head[i].index.min()
    tmax[i] = head[i].index.max()
    
df['tmin'] = df.index.map(tmin)
df['tmax'] = df.index.map(tmax)
df['median'] = df.index.map(median) 
df = df.drop(['lat', 'lon', 'uuid_hand', 'uuid_diver'], axis=1)
df['id']=df.index
df = df[['x',
         'y',
         'bkf',
         'okf',
         'median',
         'surface_level',
         'tmin',
         'tmax',
         'buis',
         'filt',
         'id']]

Save to csv

In [6]:
folder = r'.\export\csv'
if not os.path.exists(folder):
    os.makedirs(folder)
    
df.to_csv(os.path.join(folder, 'metadata.csv'), index=False)

for k, v in head.items():
    v.to_csv(os.path.join(folder, '{}.csv'.format(k)), header=True)

Save to imod

In [7]:
import imod

folder = r'.\export\imod'
if not os.path.exists(folder):
    os.makedirs(folder)

df_imod = df[['x',
              'y',
              'bkf',
              'okf',
              'median',
              'surface_level',
              'buis',
              'filt',
              'id']].copy()
df_imod.loc[:, 'id'] = 'txt\\' + df_imod.loc[:, 'id']
imod.ipf.write(os.path.join(folder, 'peilbuizen.ipf'), df_imod, indexcolumn=9)

folder = r'.\export\imod\txt'
if not os.path.exists(folder):
    os.makedirs(folder)

for k, v in head.items():
    h = v.groupby(v.index.strftime('%Y%m%d')).mean()
    header = '{}\n2\ndate,-9999\nhead_nap,-9999.0\n'.format(len(h))
    f = open(os.path.join(folder, '{}.txt'.format(k)), 'w')
    f.write(header)
    f.close()
    h.to_csv(os.path.join(folder, '{}.txt'.format(k)), mode='a')

Save image

In [8]:
import matplotlib.pyplot as plt

folder = r'.\export\image'
if not os.path.exists(folder):
    os.makedirs(folder)

for k, v in head.items():
    fig, ax = plt.subplots(figsize=[16,9])
    v.plot(ax=ax)
    ax.set_title(k)
    plt.savefig(os.path.join(folder, '{}.png'.format(k)))
    plt.close()

Save to shape

In [9]:
import geopandas, shapely.geometry

folder = r'.\export\shp'
if not os.path.exists(folder):
    os.makedirs(folder)
    
geometry = df.apply(lambda r: shapely.geometry.Point(r['x'], r['y']), axis=1)
gdf = geopandas.GeoDataFrame(df.copy(), geometry=geometry, crs='EPSG:28992')
gdf['tmin'] = gdf['tmin'].astype(str)
gdf['tmax'] = gdf['tmax'].astype(str)
gdf.to_file(os.path.join(folder, 'peilbuizen.shp'))

Save to menyanthes

In [10]:
import time

folder = r'.\export\menyanthes'
if not os.path.exists(folder):
    os.makedirs(folder)

f = open(os.path.join(folder, 'peilbuizen.csv'), 'w')

f.write('TITEL: Lizard Menyanthes Export\n')
f.write('GEBRUIKERSNAAM: {}\n'.format(os.getlogin()))
f.write('PERIODE: {}-{}\n'.format(df['tmin'].min(), df['tmax'].max())) #niet lekker
f.write('DATUM:{}\n'.format(time.ctime()))
f.write('REFERENTIE: NAP\n')

f.write('\nLOCATIE,FILTERNUMMER,EXTERNE AANDUIDING,X COORDINAAT,Y COORDINAAT,MAAIVELD NAP,GESCHAT,MEETPUNT NAP,BOVENKANT FILTER,ONDERKANT FILTER,START DATUM,EINDE DATUM\n')
for i, r in df.iterrows():
    f.write('{},{},n/a,{},{},{},,,{},{},{},{}\n'.format(r['buis'],
                                                                    r['filt'],
                                                                    round(r['x'],2),
                                                                    round(r['y'],2),
                                                                    round(r['surface_level'],2),
                                                                    round(r['bkf'],2),
                                                                    round(r['okf'],2),
                                                                    r['tmin'].strftime('%Y/%m/%d'),
                                                                    r['tmax'].strftime('%Y/%m/%d')))

f.write('\nLOCATIE,FILTERNUMMER,PEIL DATUM TIJD,STAND (NAP),BIJZONDERHEID\n')
for k,v in head.items():
    buis = k[:-3]
    filt = int(k[-3:])
    dv=v.to_frame()
    dv['dt']=dv.index.strftime('%Y/%m/%d %H:%M:%S')
    dv.apply(lambda r: f.write('{},{},{},{},\n'.format(buis, filt, r['dt'], r['head'])), axis=1)
f.close()